In [1]:
import pandas as pd
import math
import numpy as np

import heapq
import numba as nb
from scipy.stats.mstats import gmean, hmean
from state import*

# data_full = pd.read_csv('data_test.csv')
# data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
# all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])

data_full

c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[1] = 1
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[2] = 'NOT_INVEST'
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[4:] = np.full(len(data_full.columns) - 4, 1)
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:28: FutureWarning

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,90,1.000000,NOT_INVEST,HOSE,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.0,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
1,90,0.000000,AAA,HOSE,4.893114e+12,0.0,3.926632e+09,5.918750e+10,5156710.0,6.215189e+10,...,1.018940e+12,9.794080e+11,2.943622e+10,3.430300e+11,2.816613e+11,3.589115e+10,3.873387e+09,1.870343e+10,5.338777e+12,-1.579764e+12
2,90,0.000000,AAT,HOSE,8.485598e+11,1384.0,1.576288e+08,5.314787e+09,1027330.0,9.241194e+07,...,3.328005e+10,1.410469e+10,2.253305e+10,1.191474e+11,2.649353e+06,5.314787e+09,0.000000e+00,0.000000e+00,3.428906e+11,-1.813202e+11
3,90,0.000000,ABS,HOSE,9.451310e+11,0.0,6.931579e+08,1.141712e+10,737770.0,8.641370e+07,...,2.160073e+10,1.738495e+11,2.044294e+10,2.653042e+10,1.491120e+09,0.000000e+00,0.000000e+00,2.410694e+09,7.281878e+11,-7.332759e+09
4,90,0.000000,ACC,HOSE,1.911000e+12,258.0,9.234700e+04,5.818817e+09,324230.0,2.771447e+10,...,7.737720e+11,8.828534e+10,8.608350e+09,1.338862e+10,8.726839e+10,5.818817e+09,0.000000e+00,1.173808e+10,8.545696e+11,-1.158529e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,29,0.829231,GIL,HOSE,2.957500e+11,0.0,1.178573e+07,4.434805e+08,24353.0,1.918037e+08,...,5.592835e+09,2.042547e+10,1.471138e+08,1.327977e+10,9.091083e+08,2.843084e+08,0.000000e+00,2.477158e+08,4.939735e+10,-1.132837e+10
7120,29,0.792308,VSH,HOSE,8.125000e+12,0.0,0.000000e+00,4.036377e+09,80405.0,1.217429e+10,...,1.117020e+11,7.003186e+09,4.384513e+09,5.236953e+10,3.844896e+08,3.997361e+09,0.000000e+00,3.763020e+09,5.471286e+11,-1.213501e+12
7121,29,0.741068,FPT,HOSE,2.961458e+13,2550.0,6.714888e+09,1.364417e+10,101680.0,1.010007e+10,...,5.111802e+11,4.444050e+11,1.211761e+11,4.029315e+11,7.813338e+10,9.609989e+09,0.000000e+00,2.484622e+10,1.477624e+12,-2.382139e+11
7122,29,0.724551,GMD,HOSE,7.643971e+12,0.0,1.093278e+07,9.889580e+09,132220.0,2.829137e+10,...,1.335678e+12,0.000000e+00,0.000000e+00,5.367310e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.744527e+11,-2.710334e+11


In [2]:
LIST_RANK_NOT_INVEST

array([ 18,  44,   8,   2,  10,   6,   4,  19,  67,  86,  14,  86,  23,
        13,  50,   9,  13,  25,  16,  48,   8,  10,  47,  22,  30,  36,
        81,  48,  53,  64,  29,  28,  64,  47,  36,  71,  90,  77,  50,
        73,  80,  42,  93,  74,  38,  53,  46,  70,  67,  74,  25,  39,
        87, 122, 124, 134, 101, 174, 102,  71,  55,   1], dtype=int64)

In [3]:
result, per_file = normal_main(player_random1, all_fomula,  5,[0])
result, per_file

(array([3., 2.]), [0])

In [6]:
LIST_RANK_CT1, LIST_COMP1 = get_in4_fomula('(-A+T*Y/E)/A')
LIST_RANK_CT2, LIST_COMP2 = get_in4_fomula('(+N*O*X*D)/A/A/A')
data_full['(-A+T*Y/E)/A'] = np.nan_to_num(eval('(-A+T*Y/E)/A'), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
data_full['(+N*O*X*D)/A/A/A'] = np.nan_to_num(eval('(+N*O*X*D)/A/A/A'), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
